<a href="https://colab.research.google.com/github/Saqib-Mahmood/Hand_Tracking_Gesture_Recognition/blob/master/Hand_Tracking_Gesture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


file_id = '1-1S8nztX67L6SYTCU2YX1Ao3hN3UuK0Q'
destination = '/content/jdata.zip'
download_file_from_google_drive(file_id, destination)


try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass
import tensorflow as tf

from keras.preprocessing import image as kerasimg
import cv2



!pip install patool
!mkdir /content/handtracking
#if there is an error run above cell
import patoolib
patoolib.extract_archive("/content/jdata.zip", outdir="/content/jamila1/")
patoolib.extract_archive("/content/jamila1/handT.zip", outdir="/content/handtracking/")

TensorFlow 1.x selected.


Using TensorFlow backend.


     |████████████████████████████████| 81kB 5.3MB/s 
patool: Extracting /content/jdata.zip ...
patool: running /usr/bin/7z x -o/content/jamila1/ -- /content/jdata.zip
patool: ... /content/jdata.zip extracted to `/content/jamila1/'.
patool: Extracting /content/jamila1/handT.zip ...
patool: running /usr/bin/7z x -o/content/handtracking/ -- /content/jamila1/handT.zip
patool: ... /content/jamila1/handT.zip extracted to `/content/handtracking/'.


'/content/handtracking/'

In [2]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Label map utility functions."""

import logging

import tensorflow as tf
from google.protobuf import text_format
from handtracking.protos import string_int_label_map_pb2


def _validate_label_map(label_map):
    """Checks if a label map is valid.

    Args:
      label_map: StringIntLabelMap to validate.

    Raises:
      ValueError: if label map is invalid.
    """
    for item in label_map.item:
        if item.id < 1:
            raise ValueError('Label map ids should be >= 1.')


def create_category_index(categories):
    """Creates dictionary of COCO compatible categories keyed by category id.

    Args:
      categories: a list of dicts, each of which has the following keys:
        'id': (required) an integer id uniquely identifying this category.
        'name': (required) string representing category name
          e.g., 'cat', 'dog', 'pizza'.

    Returns:
      category_index: a dict containing the same entries as categories, but keyed
        by the 'id' field of each category.
    """
    category_index = {}
    for cat in categories:
        category_index[cat['id']] = cat
    return category_index


def convert_label_map_to_categories(label_map,
                                    max_num_classes,
                                    use_display_name=True):
    """Loads label map proto and returns categories list compatible with eval.

    This function loads a label map and returns a list of dicts, each of which
    has the following keys:
      'id': (required) an integer id uniquely identifying this category.
      'name': (required) string representing category name
        e.g., 'cat', 'dog', 'pizza'.
    We only allow class into the list if its id-label_id_offset is
    between 0 (inclusive) and max_num_classes (exclusive).
    If there are several items mapping to the same id in the label map,
    we will only keep the first one in the categories list.

    Args:
      label_map: a StringIntLabelMapProto or None.  If None, a default categories
        list is created with max_num_classes categories.
      max_num_classes: maximum number of (consecutive) label indices to include.
      use_display_name: (boolean) choose whether to load 'display_name' field
        as category name.  If False or if the display_name field does not exist,
        uses 'name' field as category names instead.
    Returns:
      categories: a list of dictionaries representing all possible categories.
    """
    categories = []
    list_of_ids_already_added = []
    if not label_map:
        label_id_offset = 1
        for class_id in range(max_num_classes):
            categories.append({
                'id': class_id + label_id_offset,
                'name': 'category_{}'.format(class_id + label_id_offset)
            })
        return categories
    for item in label_map.item:
        if not 0 < item.id <= max_num_classes:
            logging.info('Ignore item %d since it falls outside of requested '
                         'label range.', item.id)
            continue
        if use_display_name and item.HasField('display_name'):
            name = item.display_name
        else:
            name = item.name
        if item.id not in list_of_ids_already_added:
            list_of_ids_already_added.append(item.id)
            categories.append({'id': item.id, 'name': name})
    return categories


def load_labelmap(path):
    """Loads label map proto.

    Args:
      path: path to StringIntLabelMap proto text file.
    Returns:
      a StringIntLabelMapProto
    """
    with tf.gfile.GFile(path, 'r') as fid:
        label_map_string = fid.read()
        label_map = string_int_label_map_pb2.StringIntLabelMap()
        try:
            text_format.Merge(label_map_string, label_map)
        except text_format.ParseError:
            label_map.ParseFromString(label_map_string)
    _validate_label_map(label_map)
    return label_map


def get_label_map_dict(label_map_path):
    """Reads a label map and returns a dictionary of label names to id.

    Args:
      label_map_path: path to label_map.

    Returns:
      A dictionary mapping label names to id.
    """
    label_map = load_labelmap(label_map_path)
    label_map_dict = {}
    for item in label_map.item:
        label_map_dict[item.name] = item.id
    return label_map_dict


In [3]:
%cd handtracking/

import numpy as np
import sys
import tensorflow as tf
import os
from threading import Thread
from datetime import datetime
import cv2
from collections import defaultdict


detection_graph = tf.Graph()
sys.path.append("..")

# score threshold for showing bounding boxes.
_score_thresh = 0.27

MODEL_NAME = 'hand_inference_graph'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(MODEL_NAME, 'hand_label_map.pbtxt')

NUM_CLASSES = 1
# load label map
label_map = load_labelmap(PATH_TO_LABELS)
categories = convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = create_category_index(categories)


# Load a frozen infrerence graph into memory
def load_inference_graph():

    # load frozen tensorflow model into memory
    print("> ====== loading HAND frozen graph into memory")
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
        sess = tf.Session(graph=detection_graph)
    print(">  ====== Hand Inference graph loaded.")
    return detection_graph, sess


# draw the detected bounding boxes on the images
# You can modify this to also draw a label.
def draw_box_on_image(num_hands_detect, score_thresh, scores, boxes, im_width, im_height, image_np):
    for i in range(num_hands_detect):
        if (scores[i] > score_thresh):
            (left, right, top, bottom) = (boxes[i][1] * im_width, boxes[i][3] * im_width,
                                          boxes[i][0] * im_height, boxes[i][2] * im_height)
            p1 = (int(left), int(top))
            p2 = (int(right), int(bottom))
            cv2.rectangle(image_np, p1, p2, (77, 255, 9), 3, 1)


# Show fps value on image.
def draw_fps_on_image(fps, image_np):
    cv2.putText(image_np, fps, (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (77, 255, 9), 2)


# Actual detection .. generate scores and bounding boxes given an image
def detect_objects(image_np, detection_graph, sess):
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name(
        'detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name(
        'detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name(
        'detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name(
        'num_detections:0')

    image_np_expanded = np.expand_dims(image_np, axis=0)

    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores,
            detection_classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})
    return np.squeeze(boxes), np.squeeze(scores)


# Code to thread reading camera input.
# Source : Adrian Rosebrock
# https://www.pyimagesearch.com/2017/02/06/faster-video-file-fps-with-cv2-videocapture-and-opencv/
class WebcamVideoStream:
    def __init__(self, src, width, height):
        # initialize the video camera stream and read the first frame
        # from the stream
        self.stream = cv2.VideoCapture(src)
        self.stream.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        self.stream.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        (self.grabbed, self.frame) = self.stream.read()

        # initialize the variable used to indicate if the thread should
        # be stopped
        self.stopped = False

    def start(self):
        # start the thread to read frames from the video stream
        Thread(target=self.update, args=()).start()
        return self

    def update(self):
        # keep looping infinitely until the thread is stopped
        while True:
            # if the thread indicator variable is set, stop the thread
            if self.stopped:
                return

            # otherwise, read the next frame from the stream
            (self.grabbed, self.frame) = self.stream.read()

    def read(self):
        # return the frame most recently read
        return self.frame

    def size(self):
        # return size of the capture device
        return self.stream.get(3), self.stream.get(4)

    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True


graph , sess = load_inference_graph()
%cd ..

model = tf.keras.models.load_model('/content/jamila1/modelNew80_150size/gRec')




ClassesNames = {
        0: "Hold",
        1: "Prepa",
        2: "Return",
        3: "Stroke"
    }

def WHOISTHIS(classes):
  max1=max(classes[0])
  class1=0
  for i in range(len(classes[0])):
    if(classes[0][i]==max1):
      class1=i

  return ClassesNames[class1]



def who_is(path, DiffSum):
  img = kerasimg.load_img(path, target_size=(150, 150))

  x = kerasimg.img_to_array(img)
  x=x/255
  x = np.expand_dims(x, axis=0)


  # we map diff sum to 0.6 which means
  # we give 0.6 weightage to sequence and 0.4 to our cnn out put for prepa and return only

  seqWeightage = 0.9
  maxSumValue = 80
  Divider = 60





  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  #print('probablity:', max(classes[0]))

  if(DiffSum>0):
    probability= DiffSum/Divider
    classes[0][1] = (classes[0][1] *(1-seqWeightage)) + probability
  else:
    probability = -DiffSum/Divider
    classes[0][2] = (classes[0][2] *(1-seqWeightage)) + probability

  return WHOISTHIS(classes), True

/content/handtracking
> ====== loading HAND frozen graph into memory
>  ====== Hand Inference graph loaded.
/content
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


#testing

In [4]:
import math
!mkdir /content/videoTest
!mkdir /content/temp


leftHandList = []
rightHandList = []
lengthofList = 6
loopCounter = 0


for i in range(lengthofList):
  leftHandList.append((0,0))
  rightHandList.append((0,0))

def mainTainList( scores, boxes, im_width, im_height, loopCounter):

  
  threshHold = 0.4
  if(scores[0]>threshHold):
    hand_detected = 1
  else:
    hand_detected = 0
  if(scores[1]>threshHold):
    hand_detected = 2

  if(hand_detected == 2):

    if(int(box[0][1] * im_width) > int(box[1][1] * im_width)):
      
      left = int(boxes[0][1] * im_width)
      if(left>300 and left < 900):
        leftHandList[loopCounter] =  (int(boxes[0][1] * im_width), int(boxes[0][0] * im_height))
      else:
        leftHandList[loopCounter] = leftHandList[loopCounter-1]
                                                 
      left = int(boxes[1][1] * im_width)
      if(left>300 and left < 900):
        rightHandList[loopCounter] = (int(boxes[1][1] * im_width), int(boxes[1][0] * im_height))
      else:
        rightHandList[loopCounter] = rightHandList[loopCounter-1]

    else:
      left = int(boxes[1][1] * im_width)
      if(left>300 and left < 900):
        leftHandList[loopCounter] = (int(boxes[1][1] * im_width),  int(boxes[1][0] * im_height))
      else:
        leftHandList[loopCounter] = leftHandList[loopCounter-1]
                                                 
      left = int(boxes[0][1] * im_width)
      if(left>300 and left < 900):
        rightHandList[loopCounter] = (int(boxes[0][1] * im_width), int(boxes[0][0] * im_height))
      else:
        rightHandList[loopCounter] = rightHandList[loopCounter-1]
  

  if(hand_detected == 1):
    rightDist = math.sqrt( ((rightHandList[loopCounter-1][0]-int(boxes[0][1] * im_width) )**2) + ((rightHandList[loopCounter-1][1]-int(boxes[0][0] * im_height) )**2))
    leftDist = math.sqrt( (( leftHandList[loopCounter-1][0]-int(boxes[0][1] * im_width) )**2) + (( leftHandList[loopCounter-1][1]-int(boxes[0][0] * im_height) )**2))
    if(leftDist>rightDist):
      left = int(boxes[0][1] * im_width)
      if(left>300 and left < 900):
        rightHandList[loopCounter] = (int(boxes[0][1] * im_width), int(boxes[0][0] * im_height))
      else:
        rightHandList[loopCounter] = rightHandList[loopCounter-1]
      leftHandList[loopCounter] = leftHandList[loopCounter-1]

    else:
      left = int(boxes[0][1] * im_width)
      if(left>300 and left < 900):
        leftHandList[loopCounter] = (int(boxes[0][1] * im_width), int(boxes[0][0] * im_height))
      else:
        leftHandList[loopCounter] = leftHandList[loopCounter-1]
      rightHandList[loopCounter] = rightHandList[loopCounter-1]

  if(hand_detected == 0):
    rightHandList[loopCounter] = rightHandList[loopCounter-1]
    leftHandList[loopCounter] = leftHandList[loopCounter-1]



  loopCounter+=1
  if(loopCounter>=lengthofList):
    loopCounter=0

  return loopCounter

rightHandDiff = []
leftHandDiff  = []

for i in range(lengthofList-1):
  leftHandDiff.append(0)
  rightHandDiff.append(0)

def diffPos(loopCounter):
  rightDiffSum = 0
  leftDiffSum = 0
  ab = loopCounter+1
  if(ab>=lengthofList):
    ab=0
  for i in range(lengthofList-1):
    rightHandDiff[i] = rightHandList[loopCounter][1] - rightHandList[ab][1]
    leftHandDiff[i]  = leftHandList[loopCounter][1]  - leftHandList[ab][1]
    loopCounter+=1
    ab+=1
    if(loopCounter>=lengthofList):
      loopCounter=0
    if(ab>=lengthofList):
      ab=0

  for i in range(lengthofList-1):
    leftDiffSum+=leftHandDiff[i]
    rightDiffSum+=rightHandDiff[i]
  
    
  

  return leftDiffSum, rightDiffSum

def draw_box_label_on_image(num_hands_detect, score_thresh, scores, boxes, im_width, im_height, image_np, leftDiffSum, rightDiffSum, loopCounter):
    ex = 80
    seq = 0
    threshHold = score_thresh
    if(scores[0]>threshHold):
        hand_detected = 1
    else:
        hand_detected = 0
    if(scores[1]>threshHold):
        hand_detected = 2

    if(hand_detected == 2):
        if(int(box[0][1] * im_width) > int(box[1][1] * im_width)):
            seq =1 #first left then right
        else:
            seq = 2

    if(hand_detected == 1):
        rightDist = math.sqrt( ((rightHandList[loopCounter-1][0]-int(boxes[0][1] * im_width) )**2) + ((rightHandList[loopCounter-1][1]-int(boxes[0][0] * im_height) )**2))
        leftDist = math.sqrt( (( leftHandList[loopCounter-1][0]-int(boxes[0][1] * im_width) )**2) + (( leftHandList[loopCounter-1][1]-int(boxes[0][0] * im_height) )**2))
        if(leftDist>rightDist):
            seq = 3 # right hand first
        else:
            seq = 4
    counter = 1
    for i in range(4):
        if (scores[i] > score_thresh):

            (left, right, top, bottom) = (int(boxes[i][1] * im_width), int(boxes[i][3] * im_width),
                                          int(boxes[i][0] * im_height), int(boxes[i][2] * im_height))
            
            
            left1 = max(left-ex, 0)
            top1 = max(top-ex, 0)
            bottom1 = min(bottom + ex, im_height)
            right1 = min(right + ex, im_width)
            frame = image[top1:bottom1, left1:right1]
            cv2.imwrite('/content/temp/img1.jpg',frame)

            if(seq == 1):
                if(counter == 1):
                    category, flag = who_is('/content/temp/img1.jpg', leftDiffSum)
                    counter == 2
                else:
                    category, flag = who_is('/content/temp/img1.jpg', rightDiffSum)
            if(seq == 2):
                if(counter == 1):
                    category, flag = who_is('/content/temp/img1.jpg', rightDiffSum)
                    counter = 2
                else:
                    category, flag = who_is('/content/temp/img1.jpg', leftDiffSum)
                    
            if(seq == 3):
                category, flag = who_is('/content/temp/img1.jpg', rightDiffSum)
            if(seq == 4):
                category, flag = who_is('/content/temp/img1.jpg', leftDiffSum)

            p1 = (left, top)
            p2 = (right, bottom)
            if(left>300 and left < 900):
              cv2.rectangle(image_np, p1, p2, (77, 255, 9), 3, 3)
              cv2.putText(image_np,category+str(p1) ,p1,cv2.FONT_HERSHEY_TRIPLEX,1,(0,0,0),1)
              cv2.putText(image_np,str(leftDiffSum) ,(100,100),cv2.FONT_HERSHEY_TRIPLEX,1,(0,0,0),1)
              cv2.putText(image_np,str(rightDiffSum) ,(100,200),cv2.FONT_HERSHEY_TRIPLEX,1,(0,0,0),1)
              



#start video Testig, give inupt and output file name

In [5]:
vid_path = '/content/jamila1/1.mp4'
pathOut = '/content/1.avi'
start = 0
skipFrame = 3
# skip frame = 1 means no frame will be skipped

count = 1 #counter to keep track frames in videos
fPath = []


vidcap = cv2.VideoCapture(vid_path)
success,image = vidcap.read()

length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Total Frames: %d" %length)


while success:
    dPath = '/content/videoTest/' + 'frame%07d.jpg' % count
    fPath.append(dPath)
    cv2.imwrite(dPath, image)     # save frame as JPEG file
    #f_path.append(dst_path + "/" + position + "/"+ "frame%07d.jpg" % start)
    for jk in range(skipFrame-1):
      success,image = vidcap.read()
      count += 1
    count += 1
    success,image = vidcap.read()
    start+=1
    if(start%300==0):
      print(start , '/', int(length/skipFrame))
    #if(count>4000):
     #


frame_array = []
a=0
b=0
leftDiffSum = 0
rightDiffSum = 0

path = fPath[0]
image = cv2.imread(path)
height, width, layers = image.shape
size = (width,height)


for i in range(int(len(fPath))):
  path = fPath[a]
  a=a+1
  if(os.path.exists(path)):
    img = kerasimg.load_img(path)
    image = cv2.imread(path)
    box, scores = detect_objects(img, graph, sess)
    x1= kerasimg.img_to_array(img)
    im_width = x1.shape[1]
    im_height = x1.shape[0]

    b = mainTainList( scores, box, im_width, im_height, b)
    leftDiffSum, rightDiffSum = diffPos(b)


    draw_box_label_on_image(len(scores), 0.4, scores, box, im_width, im_height, image, leftDiffSum, rightDiffSum, b)
 
    frame_array.append(image)
    
    if(i%100==0):
      print(i,'/',len(fPath))
 


Total Frames: 224
0 / 74


In [6]:

fps = 20
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
  out.write(frame_array[i])
out.release()    

!ffmpeg -i /content/1.avi /content/y1.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [7]:
# copy มา 1 ไฟล์ มาไว้ที่ directory ปัจจุบัน
!cp /usr/local/lib/python3.6/dist-packages/imageio/resources/images/\
realshort.mp4 video.mp4

# แปลงไฟล์เป็น string ในรูปแบบ data url
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/y1.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()



# แสดงผลด้วย vdo tag    ใครงง ก็ไปเรียน HTML ใน Sololearn ได้
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [27]:
vid_path = '/content/jamila1/2.mp4'
pathOut = '/content/2.avi'
start = 0
skipFrame = 3
# skip frame = 1 means no frame will be skipped

count = 1 #counter to keep track frames in videos
fPath = []


vidcap = cv2.VideoCapture(vid_path)
success,image = vidcap.read()

length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Total Frames: %d" %length)


while success:
    dPath = '/content/videoTest/' + 'frame%07d.jpg' % count
    fPath.append(dPath)
    cv2.imwrite(dPath, image)     # save frame as JPEG file
    #f_path.append(dst_path + "/" + position + "/"+ "frame%07d.jpg" % start)
    for jk in range(skipFrame-1):
      success,image = vidcap.read()
      count += 1
    count += 1
    success,image = vidcap.read()
    start+=1
    if(start%300==0):
      print(start , '/', int(length/skipFrame))
    #if(count>4000):
     #


frame_array = []
a=0
b=0
leftDiffSum = 0
rightDiffSum = 0

path = fPath[0]
image = cv2.imread(path)
height, width, layers = image.shape
size = (width,height)


for i in range(int(len(fPath))):
  path = fPath[a]
  a=a+1
  if(os.path.exists(path)):
    img = kerasimg.load_img(path)
    image = cv2.imread(path)
    box, scores = detect_objects(img, graph, sess)
    x1= kerasimg.img_to_array(img)
    im_width = x1.shape[1]
    im_height = x1.shape[0]

    b = mainTainList( scores, box, im_width, im_height, b)
    leftDiffSum, rightDiffSum = diffPos(b)


    draw_box_label_on_image(len(scores), 0.4, scores, box, im_width, im_height, image, leftDiffSum, rightDiffSum, b)
 
    frame_array.append(image)
    
    if(i%100==0):
      print(i,'/',len(fPath))
 


Total Frames: 149
0 / 49


In [28]:

fps = 20
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
  out.write(frame_array[i])
out.release()    

!ffmpeg -i /content/2.avi /content/y2.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [29]:
# copy มา 1 ไฟล์ มาไว้ที่ directory ปัจจุบัน
!cp /usr/local/lib/python3.6/dist-packages/imageio/resources/images/\
realshort.mp4 video.mp4

# แปลงไฟล์เป็น string ในรูปแบบ data url
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/y2.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()



# แสดงผลด้วย vdo tag    ใครงง ก็ไปเรียน HTML ใน Sololearn ได้
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [30]:
vid_path = '/content/jamila1/3.mp4'
pathOut = '/content/3.avi'
start = 0
skipFrame = 3
# skip frame = 1 means no frame will be skipped

count = 1 #counter to keep track frames in videos
fPath = []


vidcap = cv2.VideoCapture(vid_path)
success,image = vidcap.read()

length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Total Frames: %d" %length)


while success:
    dPath = '/content/videoTest/' + 'frame%07d.jpg' % count
    fPath.append(dPath)
    cv2.imwrite(dPath, image)     # save frame as JPEG file
    #f_path.append(dst_path + "/" + position + "/"+ "frame%07d.jpg" % start)
    for jk in range(skipFrame-1):
      success,image = vidcap.read()
      count += 1
    count += 1
    success,image = vidcap.read()
    start+=1
    if(start%300==0):
      print(start , '/', int(length/skipFrame))
    #if(count>4000):
     #


frame_array = []
a=0
b=0
leftDiffSum = 0
rightDiffSum = 0

path = fPath[0]
image = cv2.imread(path)
height, width, layers = image.shape
size = (width,height)


for i in range(int(len(fPath))):
  path = fPath[a]
  a=a+1
  if(os.path.exists(path)):
    img = kerasimg.load_img(path)
    image = cv2.imread(path)
    box, scores = detect_objects(img, graph, sess)
    x1= kerasimg.img_to_array(img)
    im_width = x1.shape[1]
    im_height = x1.shape[0]

    b = mainTainList( scores, box, im_width, im_height, b)
    leftDiffSum, rightDiffSum = diffPos(b)


    draw_box_label_on_image(len(scores), 0.4, scores, box, im_width, im_height, image, leftDiffSum, rightDiffSum, b)
 
    frame_array.append(image)
    
    if(i%100==0):
      print(i,'/',len(fPath))
 


Total Frames: 129
0 / 42


In [31]:

fps = 20
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
  out.write(frame_array[i])
out.release()    

!ffmpeg -i /content/3.avi /content/y3.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [32]:
# copy มา 1 ไฟล์ มาไว้ที่ directory ปัจจุบัน
!cp /usr/local/lib/python3.6/dist-packages/imageio/resources/images/\
realshort.mp4 video.mp4

# แปลงไฟล์เป็น string ในรูปแบบ data url
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/y3.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()



# แสดงผลด้วย vdo tag    ใครงง ก็ไปเรียน HTML ใน Sololearn ได้
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)